## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,lebu,CL,-37.6167,-73.6500,51.66,83,3,7.54,clear sky
1,1,qaanaaq,GL,77.4840,-69.3632,-3.03,84,100,5.79,overcast clouds
2,2,souillac,MU,-20.5167,57.5167,75.58,94,75,1.14,broken clouds
3,3,hermanus,ZA,-34.4187,19.2345,62.51,87,100,2.28,overcast clouds
4,4,bluff,NZ,-46.6000,168.3333,59.81,93,91,15.82,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 74
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,souillac,MU,-20.5167,57.5167,75.58,94,75,1.14,broken clouds
9,9,ternate,ID,0.8000,127.4000,81.30,78,100,1.54,overcast clouds
12,12,saint george,US,37.1041,-113.5841,81.79,16,0,8.01,clear sky
15,15,saint-philippe,RE,-21.3585,55.7679,76.59,85,77,4.05,broken clouds
17,17,isangel,VU,-19.5500,169.2667,82.60,69,56,7.49,broken clouds
20,20,lazaro cardenas,MX,17.9583,-102.2000,83.71,52,0,11.45,clear sky
21,21,bengkulu,ID,-3.8004,102.2655,76.15,87,99,7.83,overcast clouds
22,22,shwebo,MM,22.5667,95.7000,80.04,54,68,2.04,broken clouds
24,24,rikitea,PF,-23.1203,-134.9692,78.71,71,3,19.75,clear sky
27,27,atuona,PF,-9.8000,-139.0333,79.57,74,43,17.78,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                208
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nan_value = "NaN"
preferred_cities_df.replace("", nan_value, inplace=True)
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
preferred_cities_df.count()

C:\Users\Family\.conda\envs\Pythondata\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\Family\.conda\envs\Pythondata\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                208
City                   208
Country                208
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_df = preferred_cities_df
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,souillac,MU,75.58,broken clouds,-20.5167,57.5167,
9,ternate,ID,81.30,overcast clouds,0.8000,127.4000,
12,saint george,US,81.79,clear sky,37.1041,-113.5841,
15,saint-philippe,RE,76.59,broken clouds,-21.3585,55.7679,
17,isangel,VU,82.60,broken clouds,-19.5500,169.2667,
20,lazaro cardenas,MX,83.71,clear sky,17.9583,-102.2000,
21,bengkulu,ID,76.15,overcast clouds,-3.8004,102.2655,
22,shwebo,MM,80.04,broken clouds,22.5667,95.7000,
24,rikitea,PF,78.71,clear sky,-23.1203,-134.9692,
27,atuona,PF,79.57,scattered clouds,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,souillac,MU,75.58,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
9,ternate,ID,81.30,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
12,saint george,US,81.79,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
15,saint-philippe,RE,76.59,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
17,isangel,VU,82.60,broken clouds,-19.5500,169.2667,Tanna Lodge
20,lazaro cardenas,MX,83.71,clear sky,17.9583,-102.2000,HOTEL QUINTA ANTIGUA
21,bengkulu,ID,76.15,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
22,shwebo,MM,80.04,broken clouds,22.5667,95.7000,MOON SHINE Guest House
24,rikitea,PF,78.71,clear sky,-23.1203,-134.9692,People ThankYou
27,atuona,PF,79.57,scattered clouds,-9.8000,-139.0333,Villa Enata


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))